In [1]:
import pandas as pd

In [1]:
encode_type = 'ISO-8859-1'

hello world


In [2]:
metrics_df = pd.read_csv('Metrics.csv', encoding=encoding_type)
recordtype_df = pd.read_csv('RecordType.csv', encoding=encoding_type)
subactivity_df = pd.read_csv('Component_Sub_Activity__c.csv', encoding=encoding_type)
activity_df = pd.read_csv('Component_Activity__c.csv', encoding=encoding_type)
component_df = pd.read_csv('Components__c.csv', encoding=encoding_type)
opportunity_df = pd.read_csv('Opportunity.csv', encoding=encoding_type)
account_df = pd.read_csv('Account.csv', encoding=encoding_type)

NameError: name 'encoding_type' is not defined

In [286]:
# Add RecordType to Metrics
metrics_df['RecordType'] = metrics_df['RecordTypeId'].apply(lambda x: recordtype_df.loc[recordtype_df['Id'].str.startswith(x), 'Name'].values[0])

In [287]:
# Add Organization Name to Metrics
metrics_df.drop(columns=['Name', 'Id'], inplace=True)
metrics_df = pd.merge(metrics_df, opportunity_df[['Id', 'AccountId']], left_on='Opportunity__c', right_on='Id', how='left')
metrics_df.drop(columns=['Id'], inplace=True)
metrics_df = pd.merge(metrics_df, account_df[['Id', 'Name']], left_on='AccountId', right_on='Id', how='left')
metrics_df = metrics_df.drop(columns=['AccountId', 'Id'])
metrics_df = metrics_df.rename(columns={'Name': 'Organization Name'})

In [288]:
# Add Partner Organization Name to Metrics
metrics_df = pd.merge(metrics_df, account_df[['Id', 'Name']], left_on='Partner_Organization__c', right_on='Id', how='left')
metrics_df = metrics_df.drop(columns=['Id'])
metrics_df = metrics_df.rename(columns={'Name': 'Partner Organization Name'})

In [289]:
# Add Component Name to Metrics
merged_df = pd.merge(metrics_df, subactivity_df[['Id','Activity__c' ]], left_on='Sub_Activity__c', right_on='Id')
merged_df = pd.merge(merged_df.drop(columns=['Id']), activity_df[['Id', 'Component__c']], left_on='Activity__c', right_on='Id')
merged_df = pd.merge(merged_df.drop(columns=['Id', 'Activity__c']), component_df[['Id', 'Name']], left_on='Component__c', right_on='Id')
merged_df = merged_df.drop(columns=['Component__c', 'Id'])
merged_df.rename(columns={'Name': 'Component'}, inplace=True)

In [290]:
# split the metrics into different files based on component and recordtype
grouped_by_component = merged_df.groupby('Component')

for component, group in grouped_by_component:
    grouped_by_recordtype = group.groupby('RecordType')

    for recordtype, group in grouped_by_recordtype:
        # remove all empty columns
        group = group.dropna(axis=1, how='all')
        filename = f'outputs/{component}_{recordtype}.csv'
        group.to_csv(filename, index=False)

In [ ]:
agents_data = {
    'Id': [1, 2, 3, 4, 5],
    'Name': ['Arame', 'Daouda', 'Souley', 'Amadou', 'Demba']
}
agents_df = pd.DataFrame(agents_data)

# Create the Practices dataframe
practices_data = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'agentId': [1, 2, 1, 3, 2, 3, 5, 2, 5],
    'practice': ['pruning', 'pruning', 'drying', 'fire belting', 'fire belting', 'intercroping', 'intercroping', 'drying', 'pruning']
}
practices_df = pd.DataFrame(practices_data)

# Merge the dataframes on agentId
merged_df = pd.merge(agents_df, practices_df, left_on='Id', right_on='agentId', how='left')

# Group by agent and aggregate the practices into a comma-separated string
result_df = merged_df.groupby(['Id', 'Name'])['practice'].apply(lambda x: ', '.join(x.dropna())).reset_index()

# Rename the 'practice' column to 'practices'
result_df.rename(columns={'practice': 'practices'}, inplace=True)

print(result_df)